In [1]:
import json
import requests
import datetime
import time

In [ ]:
env = json.loads(open("../env.txt", "r").read())
token_url = 'https://oauth.battle.net/token'

data = {'grant_type': 'client_credentials'}

response = requests.post(token_url, data=data, auth=(env["client_identity"], env["client_secret"]))
access_token = response.json().get('access_token')
conversion_maps = {}


In [3]:
def get_conversion_dict(access_token, parameter):
	api_url = f"https://us.api.blizzard.com/hearthstone/metadata/{parameter}?locale=en_US"
	headers = {'Authorization': f'Bearer {access_token}'}

	response = requests.get(api_url, headers=headers)
	response_map = {}
	if response.status_code == 200:
		response_json = response.json()
		for response in response_json:
			response_map[response["id"]] = response["name"]
	else:
		print(f"Error: {response.status_code} - {response.text}")
		return None
	return response_map

In [ ]:
# Things we care about: CLASS ID, Keyword, Rarity, Set Release, MinionType / Spell School, Type, Mana cost, Release Year
conversion_maps["classId"] = get_conversion_dict(access_token, "classes")
conversion_maps["rarityId"] = get_conversion_dict(access_token, "rarities")
conversion_maps["keywordIds"] = get_conversion_dict(access_token, "keywords")
conversion_maps["minionTypeId"] = get_conversion_dict(access_token, "minionTypes")
conversion_maps["spellSchoolId"] = get_conversion_dict(access_token, "spellSchools")
conversion_maps["cardTypeId"] = get_conversion_dict(access_token, "types")
conversion_maps["sets"] = get_conversion_dict(access_token, "sets")

with open("../id_conversions.json", "w") as outfile:
	outfile.write(json.dumps(conversion_maps, indent=4))